# Copying figures from Holst et al "Impacts of summer water limitation on the carbon balance of a Scots pine forest in the southern upper Rhine plain"

In [ ]:
# Settings
Username   = 'Beheerder'
years      = range(2001,2020)    #(1997,2021) # Set years to download

In [ ]:
import os
datapath   = os.path.join('../')
print('datapath is set to %s'%datapath)

# !pip install numpy
# !pip install pandas
# !pip install matplotlib
# !pip install plotly 
# !pip install cufflinks
#!pip install colorspacious
#!pip install seaborn

import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
#import plotly.express as px
#import cufflinks as cf
import matplotlib.dates as mdate
import matplotlib.ticker as ticker
from matplotlib import cm
#from colorspacious import cspace_converter
import scipy.stats as stats
#cf.go_offline()
# cf.set_config_file(offline=False, world_readable=True)

from datetime import datetime, timedelta
import sys
sys.path.insert(0, os.path.join(datapath,'PythonScripts'))
from Loobos_Toolbox import dateparse, dateparse_Gapfilled, Read_LoobosEddFinal, Read_LooStor, Read_LoodatGapfill, Read_Loobos_halfhourly, Read_Loobos_meteo, Read_Loobos_soil, Read_Loobos_profile

from Ags_model import runAgs, calc_LE

In [ ]:
from FilterData import Filter_wrap

In [ ]:
#these next two lines are to prevent re-loading the data. If you want to re-load data, instead comment them out
if not 'progress' in globals(): progress = list()
if not 'dataloaded' in progress:
  # Read files
    df_EC           = Read_LoobosEddFinal    (years,datapath)
    df_Stor         = Read_LooStor           (years,datapath)
    df_Comb         = Read_LoodatGapfill     (years,datapath)
    df_NEE          = Read_Loobos_halfhourly (years,datapath)
    df_meteo        = Read_Loobos_meteo      (years,datapath)
    df_soil         = Read_Loobos_soil       (years,datapath) 
    df_profile      = Read_Loobos_profile    (years,datapath)
    progress.append('dataloaded')

In [ ]:
#from FilterData import Filter_wrap
CO2,Locorr,VPD,Ustar,df_profile_filter,df_meteo_filter,df_Comb_filter,df_EC_filter=Filter_wrap(df_Comb,df_profile,df_meteo,df_EC,filterversion='default')
#NOTE: df_Stor is NOT FILTERED

# Start of figure generation

## Figure 2

In [ ]:
#set aspect ratio and dpi
aspectratio=21/9
fig_wid=6 #we want 6 inch width
print(fig_wid/aspectratio) # around 2.6
fig_hei=2.6
dpi=144 #can be as high as 300, standard is 72 which is way too low

In [ ]:
# plotting F_NEE(micromol /m2 /s) over Ta (deg C), Ts (deg C), VPD (hPa), separated into two classes: PAR>1000 and PAR500-1000
#reminder to self: maybe make a LUE curve to justify the classes >1000 and 500-1000? Or unnecessary, quote literature?

#F_NEE data sourced from: df_Comb: NEE
#VPD data sourced from: df_Comb: VPD
#Ta (air temp) data sourced from: df_Comb: Tair
#Ts (soil temp) data sourced from: df_Comb: Tsoil
#PAR data sourced from: df_meteo: PAR

df_fig2 = pd.concat([df_Comb_filter['NEE'],df_Comb_filter['VPD'],df_Comb_filter['Tair'],df_Comb_filter['Tsoil'], df_meteo_filter['PAR']],axis=1,sort=False)

In [ ]:
df_fig2.plot()

In [ ]:
#df_test = df_fig2.loc[df_fig2['PAR']<1000]
#df_test2 = df_test.loc[df_test['PAR']>500]

#df_test2e = df_fig2.loc[(500<df_fig2['PAR']<1000)]

#df_test2e.equals(df_test2)
#this throws error

### start of figure 2e and f

In [ ]:
#fig 2 e)

#because the marker argument in scatter() doesn't accept a long list the same size of the data where each entry specifies the marker of that point (unlike c - color arg),
#we have to split the dataset into the categories and plot it over each other with multiple instances of scatter()

#slope1, intercept1, r_value1, p_value1, std_err1 = stats.linregress(df_tmp1['sim_data_1'], df_tmp1['sim_data_2'])  #linregres x, y . note r_value is Pearson's coefficient. R^2 is r_value**2
#print('slope, intercept, R2:', slope1, intercept1, r_value1**2)

df_fig2e = df_fig2.loc[df_fig2['PAR']<1000]

#make aliases
Tair=df_fig2e['Tair']
NEE=df_fig2e['NEE']
VPD=df_fig2e['VPD']

#because the marker argument in scatter() doesn't accept a long list the same size of the data where each entry specifies the marker of that point (unlike c - color arg),
#we have to split the dataset into the categories and plot it over each other with multiple instances of scatter()

#separate data:
#important! check if lengths of each of these are the same
fig2e_cat1=df_fig2e.loc[(Tair>=10)&(Tair<=15)]
fig2e_cat2=df_fig2e.loc[(Tair>15)&(Tair<=20)]
fig2e_cat3=df_fig2e.loc[(Tair>20)&(Tair<=25)]
fig2e_cat4=df_fig2e.loc[Tair>25]

# visual reference for segments and boundaries:  |-(10-15)--|---(15-20)--|--(20-25)--|-(25+)---
# meaning 4 segments with 4 boundaries (upper segment unbounded: 25+ but lower segment bounded: 10 or greater

#colorbar settings
#for some reason the object created and assigned to cmap (matplotlib.colors.LinearSegmentedColormap) does not have an attribute .colors (which it should have? or maybe it's just meant to be a constructor?
#as such, to properly assign the 'outside range' cbar colors I'd have to either do a workaround, or I can just manually asign my colors. I'm opting for the latter.
colors=['#3A4CC0', '#AAC6FD', '#F6B79C', '#B30326']
cmap= mpl.colors.ListedColormap(colors, name='coolwarm_4', N=None)
colors_cbar=['#3A4CC0', '#AAC6FD', '#F6B79C', '#F6B79C']
cmap_cbar= mpl.colors.ListedColormap(colors_cbar, name='coolwarm_4', N=None) #have to do this weird trick to get the single-side unbounded cbar to work
cmap_cbar.set_over('#B30326')

#cols = [mpl.colors.to_hex(cmap(0)),mpl.colors.to_hex(cmap(1)),mpl.colors.to_hex(cmap(2)),mpl.colors.to_hex(cmap(3))] #may be unnecessary, use cmap(0) cmap(1) cmap(2) etc

cbar_bounds = [10, 15 ,20, 25 ] #boundaries excluding the abs min and abs max
norm = mpl.colors.BoundaryNorm(cbar_bounds, cmap.N)

fig,ax = plt.subplots()
ax_cbar = fig.add_axes([0.05, -0.20, 0.9, 0.05]) #coords are in proportion to fig size (fraction of fig dimension). 0,0 being bottom left corner, 1,1 being top right. format is [x1,y1,x2,y2] where x1,y1 is origin point of axes, and x2,y2 is the width and height of axes, respectively.

p1=ax.scatter(fig2e_cat1['VPD'],fig2e_cat1['NEE'],marker='.',s=5,c=colors[0],alpha=0.3) #scatter(x,y)
p2=ax.scatter(fig2e_cat2['VPD'],fig2e_cat2['NEE'],marker='s',s=5,c=colors[1],alpha=0.3) #scatter(x,y)
p3=ax.scatter(fig2e_cat3['VPD'],fig2e_cat3['NEE'],marker='d',s=5,c=colors[2],alpha=0.3) #scatter(x,y)
p4=ax.scatter(fig2e_cat4['VPD'],fig2e_cat4['NEE'],marker='+',s=10,c=colors[3],alpha=0.3) #scatter(x,y)

#trendlines (work in progress)
#ax.axline((0.0,intercept1),slope=slope1,c='black',label='reference') 
#ax.axline ((0.0,0.0), slope=1, c='r',linestyle='dashed',label='1:1')

ax.set_ylim(-45,30)
ax.set_xlim(-1.5,45)
ax.set_ylabel(r'NEE [$\mu molm^{-2}s^{-1}$] (?)')
ax.set_xlabel(r'VPD [Pa or hPA?]')
#fig.suptitle('Correlation of increased VPD to reference run for An, \n (2008-2017 during growth seasons May-Oct.)')
ax.set_title('Fig2e: NEE vs VPD (color by Ta), PAR<1000')
#ax.legend(loc='upper left')
#fig.subplots_adjust(top=0.87) #fix the top margin text overlap  

cb = mpl.colorbar.ColorbarBase(ax_cbar, cmap=cmap_cbar,
                                norm=norm,
                                # to use 'extend', you must specify two extra boundaries:
                                boundaries=cbar_bounds+[25],
                                extend='max', #use 'both' for left and right
                                ticks=cbar_bounds,  # optional
                                spacing='proportional',
                                orientation='horizontal')
cb.set_label('Tair')
#cbar=plt.colorbar(p1,ax=ax)
#cbar.ax.set_ylabel('Tair')
fig.set_figwidth(6)
fig.set_figheight(2.6)
fig.set_dpi(300)
plt.plot()

In [ ]:
#fig 2 f)

#slope1, intercept1, r_value1, p_value1, std_err1 = stats.linregress(df_tmp1['sim_data_1'], df_tmp1['sim_data_2'])  #linregres x, y . note r_value is Pearson's coefficient. R^2 is r_value**2
#print('slope, intercept, R2:', slope1, intercept1, r_value1**2)

df_fig2f = df_fig2.loc[(df_fig2['PAR']<1000) & (df_fig2['PAR']>500)]

#make aliases
Tair=df_fig2f['Tair']
NEE=df_fig2f['NEE']
VPD=df_fig2f['VPD']

#because the marker argument in scatter() doesn't accept a long list the same size of the data where each entry specifies the marker of that point (unlike c - color arg),
#we have to split the dataset into the categories and plot it over each other with multiple instances of scatter()

#separate data:
#important! check if lengths of each of these are the same
fig2f_cat1=df_fig2f.loc[(Tair>=10)&(Tair<=15)]
fig2f_cat2=df_fig2f.loc[(Tair>15)&(Tair<=20)]
fig2f_cat3=df_fig2f.loc[(Tair>20)&(Tair<=25)]
fig2f_cat4=df_fig2f.loc[Tair>25]

# visual reference for segments and boundaries:  |-(10-15)--|---(15-20)--|--(20-25)--|-(25+)---
# meaning 4 segments with 4 boundaries (upper segment unbounded: 25+ but lower segment bounded: 10 or greater

#colorbar settings
#for some reason the object created and assigned to cmap (matplotlib.colors.LinearSegmentedColormap) does not have an attribute .colors (which it should have? or maybe it's just meant to be a constructor?
#as such, to properly assign the 'outside range' cbar colors I'd have to either do a workaround, or I can just manually asign my colors. I'm opting for the latter.
colors=['#3A4CC0', '#AAC6FD', '#F6B79C', '#B30326']
cmap= mpl.colors.ListedColormap(colors, name='coolwarm_4', N=None)
colors_cbar=['#3A4CC0', '#AAC6FD', '#F6B79C', '#F6B79C']
cmap_cbar= mpl.colors.ListedColormap(colors_cbar, name='coolwarm_4', N=None) #have to do this weird trick to get the single-side unbounded cbar to work
cmap_cbar.set_over('#B30326')

#cols = [mpl.colors.to_hex(cmap(0)),mpl.colors.to_hex(cmap(1)),mpl.colors.to_hex(cmap(2)),mpl.colors.to_hex(cmap(3))] #may be unnecessary, use cmap(0) cmap(1) cmap(2) etc

cbar_bounds = [10, 15 ,20, 25 ] #boundaries excluding the abs min and abs max
norm = mpl.colors.BoundaryNorm(cbar_bounds, cmap.N)

fig,ax = plt.subplots()
ax_cbar = fig.add_axes([0.05, -0.20, 0.9, 0.05]) #coords are in proportion to fig size (fraction of fig dimension). 0,0 being bottom left corner, 1,1 being top right. format is [x1,y1,x2,y2] where x1,y1 is origin point of axes, and x2,y2 is the width and height of axes, respectively.

p1=ax.scatter(fig2f_cat1['VPD'],fig2f_cat1['NEE'],marker='.',s=5,c=colors[0],alpha=0.3) #scatter(x,y)
p2=ax.scatter(fig2f_cat2['VPD'],fig2f_cat2['NEE'],marker='s',s=5,c=colors[1],alpha=0.3) #scatter(x,y)
p3=ax.scatter(fig2f_cat3['VPD'],fig2f_cat3['NEE'],marker='d',s=5,c=colors[2],alpha=0.3) #scatter(x,y)
p4=ax.scatter(fig2f_cat4['VPD'],fig2f_cat4['NEE'],marker='+',s=10,c=colors[3],alpha=0.3) #scatter(x,y)

#trendlines (work in progress)
#ax.axline((0.0,intercept1),slope=slope1,c='black',label='reference') 
#ax.axline ((0.0,0.0), slope=1, c='r',linestyle='dashed',label='1:1')

ax.set_ylim(-45,30)
ax.set_xlim(-1.5,45)
ax.set_ylabel(r'NEE [$\mu molm^{-2}s^{-1}$] (?)')
ax.set_xlabel(r'VPD [Pa or hPA?]')
#fig.suptitle('Correlation of increased VPD to reference run for An, \n (2008-2017 during growth seasons May-Oct.)')
ax.set_title('Fig2f: NEE vs VPD (color by Ta), 500<PAR<1000')
#ax.legend(loc='upper left')
#fig.subplots_adjust(top=0.87) #fix the top margin text overlap  

cb = mpl.colorbar.ColorbarBase(ax_cbar, cmap=cmap_cbar,
                                norm=norm,
                                # to use 'extend', you must specify two extra boundaries:
                                boundaries=cbar_bounds+[25],
                                extend='max', #use 'both' for left and right
                                ticks=cbar_bounds,  # optional
                                spacing='proportional',
                                orientation='horizontal')
cb.set_label('Tair')
#cbar=plt.colorbar(p1,ax=ax)
#cbar.ax.set_ylabel('Tair')
fig.set_figwidth(6)
fig.set_figheight(2.6)
fig.set_dpi(300)
plt.plot()

In [ ]:
cmap = mpl.colormaps['coolwarm'].resampled(4)
#for i,c in enumerate(cmap.get_colors):
cmap(-1050)

In [ ]:
cmap

In [ ]:
colorz = [(0.2298057, 0.298717966, 0.753683153), (1, 1, 1), (0.705673158, 0.01555616, 0.150232812)]
cmapz = mpl.colors.LinearSegmentedColormap.from_list('test', colorz, N=4)
cmapz

In [ ]:
cmap = mpl.colors.ListedColormap(['#3A4CC0', '#AAC6FD', '#F6B79C', '#B30326'], name='coolwarm_4', N=None)
cmap(0)

In [ ]:
#adapted figure: plot GPP instead of NEE (thus eliminating effects respiration, as NEE = GPP - respiration (ecosys.)

#GPP data sourced from: df_Comb: GPP_f

### end of figure 2e and f

### start of figure 2a and b

In [ ]:
#fig 2 a)

#slope1, intercept1, r_value1, p_value1, std_err1 = stats.linregress(df_tmp1['sim_data_1'], df_tmp1['sim_data_2'])  #linregres x, y . note r_value is Pearson's coefficient. R^2 is r_value**2
#print('slope, intercept, R2:', slope1, intercept1, r_value1**2)

df_fig2a = df_fig2.loc[df_fig2['PAR']<1000]

#make aliases
Tair=df_fig2a['Tair']
NEE=df_fig2a['NEE']
VPD=df_fig2a['VPD']

#because the marker argument in scatter() doesn't accept a long list the same size of the data where each entry specifies the marker of that point (unlike c - color arg),
#we have to split the dataset into the categories and plot it over each other with multiple instances of scatter()

#separate data:
#important! check if lengths of each of these are the same
fig2a_cat1=df_fig2a.loc[(VPD>=0)&(VPD<=5)]
fig2a_cat2=df_fig2a.loc[(VPD>5)&(VPD<=10)]
fig2a_cat3=df_fig2a.loc[(VPD>10)&(VPD<=15)]
fig2a_cat4=df_fig2a.loc[VPD>15]

# visual reference for segments and boundaries:  |-(10-15)--|---(15-20)--|--(20-25)--|-(25+)---
# meaning 4 segments with 4 boundaries (upper segment unbounded: 25+ but lower segment bounded: 10 or greater

#colorbar settings
#for some reason the object created and assigned to cmap (matplotlib.colors.LinearSegmentedColormap) does not have an attribute .colors (which it should have? or maybe it's just meant to be a constructor?
#as such, to properly assign the 'outside range' cbar colors I'd have to either do a workaround, or I can just manually asign my colors. I'm opting for the latter.
colors=['#3A4CC0', '#AAC6FD', '#F6B79C', '#B30326']
cmap= mpl.colors.ListedColormap(colors, name='coolwarm_4', N=None)
colors_cbar=['#3A4CC0', '#AAC6FD', '#F6B79C', '#F6B79C']
cmap_cbar= mpl.colors.ListedColormap(colors_cbar, name='coolwarm_4', N=None) #have to do this weird trick to get the single-side unbounded cbar to work
cmap_cbar.set_over('#B30326')

#cols = [mpl.colors.to_hex(cmap(0)),mpl.colors.to_hex(cmap(1)),mpl.colors.to_hex(cmap(2)),mpl.colors.to_hex(cmap(3))] #may be unnecessary, use cmap(0) cmap(1) cmap(2) etc

cbar_bounds = [0, 5 ,10, 15 ] #boundaries excluding the abs min and abs max
norm = mpl.colors.BoundaryNorm(cbar_bounds, cmap.N)

fig,ax = plt.subplots()
ax_cbar = fig.add_axes([0.05, -0.20, 0.9, 0.05]) #coords are in proportion to fig size (fraction of fig dimension). 0,0 being bottom left corner, 1,1 being top right. format is [x1,y1,x2,y2] where x1,y1 is origin point of axes, and x2,y2 is the width and height of axes, respectively.

p1=ax.scatter(fig2a_cat1['Tair'],fig2a_cat1['NEE'],marker='.',s=5,c=colors[0],alpha=0.3) #scatter(x,y)
p2=ax.scatter(fig2a_cat2['Tair'],fig2a_cat2['NEE'],marker='s',s=5,c=colors[1],alpha=0.3) #scatter(x,y)
p3=ax.scatter(fig2a_cat3['Tair'],fig2a_cat3['NEE'],marker='d',s=5,c=colors[2],alpha=0.3) #scatter(x,y)
p4=ax.scatter(fig2a_cat4['Tair'],fig2a_cat4['NEE'],marker='+',s=10,c=colors[3],alpha=0.3) #scatter(x,y)

#trendlines (work in progress)
#ax.axline((0.0,intercept1),slope=slope1,c='black',label='reference') 
#ax.axline ((0.0,0.0), slope=1, c='r',linestyle='dashed',label='1:1')

ax.set_ylim(-30,20)
ax.set_xlim(-20,40)
ax.set_ylabel(r'NEE [$\mu molm^{-2}s^{-1}$] (?)')
ax.set_xlabel(r'Tair [$\degree$C]')
#fig.suptitle('Correlation of increased VPD to reference run for An, \n (2008-2017 during growth seasons May-Oct.)')
ax.set_title('Fig2a: NEE vs Tair (color by VPD), PAR<1000')
#ax.legend(loc='upper left')
#fig.subplots_adjust(top=0.87) #fix the top margin text overlap  

cb = mpl.colorbar.ColorbarBase(ax_cbar, cmap=cmap_cbar,
                                norm=norm,
                                # to use 'extend', you must specify two extra boundaries:
                                boundaries=cbar_bounds+[15],
                                extend='max', #use 'both' for left and right
                                ticks=cbar_bounds,  # optional
                                spacing='proportional',
                                orientation='horizontal')
cb.set_label('VPD')
#cbar=plt.colorbar(p1,ax=ax)
#cbar.ax.set_ylabel('Tair')
fig.set_figwidth(6)
fig.set_figheight(2.6)
fig.set_dpi(300)
plt.plot()

In [ ]:
#fig 2 b)

#slope1, intercept1, r_value1, p_value1, std_err1 = stats.linregress(df_tmp1['sim_data_1'], df_tmp1['sim_data_2'])  #linregres x, y . note r_value is Pearson's coefficient. R^2 is r_value**2
#print('slope, intercept, R2:', slope1, intercept1, r_value1**2)

df_fig2b = df_fig2.loc[(df_fig2['PAR']<1000) & (df_fig2['PAR']>500)]

#make aliases
Tair=df_fig2b['Tair']
NEE=df_fig2b['NEE']
VPD=df_fig2b['VPD']

#because the marker argument in scatter() doesn't accept a long list the same size of the data where each entry specifies the marker of that point (unlike c - color arg),
#we have to split the dataset into the categories and plot it over each other with multiple instances of scatter()

#separate data:
#important! check if lengths of each of these are the same
fig2b_cat1=df_fig2b.loc[(VPD>=0)&(VPD<=5)]
fig2b_cat2=df_fig2b.loc[(VPD>5)&(VPD<=10)]
fig2b_cat3=df_fig2b.loc[(VPD>10)&(VPD<=15)]
fig2b_cat4=df_fig2b.loc[VPD>15]

# visual reference for segments and boundaries:  |-(10-15)--|---(15-20)--|--(20-25)--|-(25+)---
# meaning 4 segments with 4 boundaries (upper segment unbounded: 25+ but lower segment bounded: 10 or greater

#colorbar settings
#for some reason the object created and assigned to cmap (matplotlib.colors.LinearSegmentedColormap) does not have an attribute .colors (which it should have? or maybe it's just meant to be a constructor?
#as such, to properly assign the 'outside range' cbar colors I'd have to either do a workaround, or I can just manually asign my colors. I'm opting for the latter.
colors=['#3A4CC0', '#AAC6FD', '#F6B79C', '#B30326']
cmap= mpl.colors.ListedColormap(colors, name='coolwarm_4', N=None)
colors_cbar=['#3A4CC0', '#AAC6FD', '#F6B79C', '#F6B79C']
cmap_cbar= mpl.colors.ListedColormap(colors_cbar, name='coolwarm_4', N=None) #have to do this weird trick to get the single-side unbounded cbar to work
cmap_cbar.set_over('#B30326')

#cols = [mpl.colors.to_hex(cmap(0)),mpl.colors.to_hex(cmap(1)),mpl.colors.to_hex(cmap(2)),mpl.colors.to_hex(cmap(3))] #may be unnecessary, use cmap(0) cmap(1) cmap(2) etc

cbar_bounds = [0, 5 ,10, 15 ] #boundaries excluding the abs min and abs max
norm = mpl.colors.BoundaryNorm(cbar_bounds, cmap.N)

fig,ax = plt.subplots()
ax_cbar = fig.add_axes([0.05, -0.20, 0.9, 0.05]) #coords are in proportion to fig size (fraction of fig dimension). 0,0 being bottom left corner, 1,1 being top right. format is [x1,y1,x2,y2] where x1,y1 is origin point of axes, and x2,y2 is the width and height of axes, respectively.

p1=ax.scatter(fig2b_cat1['Tair'],fig2b_cat1['NEE'],marker='.',s=5,c=colors[0],alpha=0.3) #scatter(x,y)
p2=ax.scatter(fig2b_cat2['Tair'],fig2b_cat2['NEE'],marker='s',s=5,c=colors[1],alpha=0.3) #scatter(x,y)
p3=ax.scatter(fig2b_cat3['Tair'],fig2b_cat3['NEE'],marker='d',s=5,c=colors[2],alpha=0.3) #scatter(x,y)
p4=ax.scatter(fig2b_cat4['Tair'],fig2b_cat4['NEE'],marker='+',s=10,c=colors[3],alpha=0.3) #scatter(x,y)

#trendlines (work in progress)
#ax.axline((0.0,intercept1),slope=slope1,c='black',label='reference') 
#ax.axline ((0.0,0.0), slope=1, c='r',linestyle='dashed',label='1:1')

ax.set_ylim(-30,20)
ax.set_xlim(-20,40)
ax.set_ylabel(r'NEE [$\mu molm^{-2}s^{-1}$] (?)')
ax.set_xlabel(r'Tair [$\degree$C]')
#fig.suptitle('Correlation of increased VPD to reference run for An, \n (2008-2017 during growth seasons May-Oct.)')
ax.set_title('Fig2b: NEE vs Tair (color by VPD), 500<PAR<1000')
#ax.legend(loc='upper left')
#fig.subplots_adjust(top=0.87) #fix the top margin text overlap  

cb = mpl.colorbar.ColorbarBase(ax_cbar, cmap=cmap_cbar,
                                norm=norm,
                                # to use 'extend', you must specify two extra boundaries:
                                boundaries=cbar_bounds+[15],
                                extend='max', #use 'both' for left and right
                                ticks=cbar_bounds,  # optional
                                spacing='proportional',
                                orientation='horizontal')
cb.set_label('VPD')
#cbar=plt.colorbar(p1,ax=ax)
#cbar.ax.set_ylabel('Tair')
fig.set_figwidth(6)
fig.set_figheight(2.6)
fig.set_dpi(300)
plt.plot()

### end of figure 2a and b

### start of figure 2c and d

In [ ]:
#fig 2 c)

#slope1, intercept1, r_value1, p_value1, std_err1 = stats.linregress(df_tmp1['sim_data_1'], df_tmp1['sim_data_2'])  #linregres x, y . note r_value is Pearson's coefficient. R^2 is r_value**2
#print('slope, intercept, R2:', slope1, intercept1, r_value1**2)

df_fig2c = df_fig2.loc[df_fig2['PAR']<1000]
#df_fig2c = df_fig2.loc[(df_fig2['PAR']<1000) & (df_fig2['PAR']>500)]

#make aliases
Tair=df_fig2c['Tair']
Tsoil=df_fig2c['Tsoil']
NEE=df_fig2c['NEE']
VPD=df_fig2c['VPD']

#because the marker argument in scatter() doesn't accept a long list the same size of the data where each entry specifies the marker of that point (unlike c - color arg),
#we have to split the dataset into the categories and plot it over each other with multiple instances of scatter()

#separate data:
#important! check if lengths of each of these are the same
fig2c_cat1=df_fig2c.loc[(VPD>=0)&(VPD<=5)]
fig2c_cat2=df_fig2c.loc[(VPD>5)&(VPD<=10)]
fig2c_cat3=df_fig2c.loc[(VPD>10)&(VPD<=15)]
fig2c_cat4=df_fig2c.loc[VPD>15]

# visual reference for segments and boundaries:  |-(10-15)--|---(15-20)--|--(20-25)--|-(25+)---
# meaning 4 segments with 4 boundaries (upper segment unbounded: 25+ but lower segment bounded: 10 or greater

#colorbar settings
#for some reason the object created and assigned to cmap (matplotlib.colors.LinearSegmentedColormap) does not have an attribute .colors (which it should have? or maybe it's just meant to be a constructor?
#as such, to properly assign the 'outside range' cbar colors I'd have to either do a workaround, or I can just manually asign my colors. I'm opting for the latter.
colors=['#3A4CC0', '#AAC6FD', '#F6B79C', '#B30326']
cmap= mpl.colors.ListedColormap(colors, name='coolwarm_4', N=None)
colors_cbar=['#3A4CC0', '#AAC6FD', '#F6B79C', '#F6B79C']
cmap_cbar= mpl.colors.ListedColormap(colors_cbar, name='coolwarm_4', N=None) #have to do this weird trick to get the single-side unbounded cbar to work
cmap_cbar.set_over('#B30326')

#cols = [mpl.colors.to_hex(cmap(0)),mpl.colors.to_hex(cmap(1)),mpl.colors.to_hex(cmap(2)),mpl.colors.to_hex(cmap(3))] #may be unnecessary, use cmap(0) cmap(1) cmap(2) etc

cbar_bounds = [0, 5 ,10, 15 ] #boundaries excluding the abs min and abs max
norm = mpl.colors.BoundaryNorm(cbar_bounds, cmap.N)

fig,ax = plt.subplots()
ax_cbar = fig.add_axes([0.05, -0.20, 0.9, 0.05]) #coords are in proportion to fig size (fraction of fig dimension). 0,0 being bottom left corner, 1,1 being top right. format is [x1,y1,x2,y2] where x1,y1 is origin point of axes, and x2,y2 is the width and height of axes, respectively.

p1=ax.scatter(fig2c_cat1['Tsoil'],fig2c_cat1['NEE'],marker='.',s=5,c=colors[0],alpha=0.3) #scatter(x,y)
p2=ax.scatter(fig2c_cat2['Tsoil'],fig2c_cat2['NEE'],marker='s',s=5,c=colors[1],alpha=0.3) #scatter(x,y)
p3=ax.scatter(fig2c_cat3['Tsoil'],fig2c_cat3['NEE'],marker='d',s=5,c=colors[2],alpha=0.3) #scatter(x,y)
p4=ax.scatter(fig2c_cat4['Tsoil'],fig2c_cat4['NEE'],marker='+',s=10,c=colors[3],alpha=0.3) #scatter(x,y)

#trendlines (work in progress)
#ax.axline((0.0,intercept1),slope=slope1,c='black',label='reference') 
#ax.axline ((0.0,0.0), slope=1, c='r',linestyle='dashed',label='1:1')

ax.set_xlim(0,20)
ax.set_ylim(-35,25)
ax.set_ylabel(r'NEE [$\mu molm^{-2}s^{-1}$] (?)')
ax.set_xlabel(r'Tsoil [$\degree$C]')
#fig.suptitle('Correlation of increased VPD to reference run for An, \n (2008-2017 during growth seasons May-Oct.)')
ax.set_title('Fig2c: NEE vs Tsoil (color by VPD), PAR<1000')
#ax.legend(loc='upper left')
#fig.subplots_adjust(top=0.87) #fix the top margin text overlap  

cb = mpl.colorbar.ColorbarBase(ax_cbar, cmap=cmap_cbar,
                                norm=norm,
                                # to use 'extend', you must specify two extra boundaries:
                                boundaries=cbar_bounds+[15],
                                extend='max', #use 'both' for left and right
                                ticks=cbar_bounds,  # optional
                                spacing='proportional',
                                orientation='horizontal')
cb.set_label('VPD')
#cbar=plt.colorbar(p1,ax=ax)
#cbar.ax.set_ylabel('Tair')
fig.set_figwidth(6)
fig.set_figheight(2.6)
fig.set_dpi(300)
plt.plot()

In [ ]:
#fig 2 d)

#slope1, intercept1, r_value1, p_value1, std_err1 = stats.linregress(df_tmp1['sim_data_1'], df_tmp1['sim_data_2'])  #linregres x, y . note r_value is Pearson's coefficient. R^2 is r_value**2
#print('slope, intercept, R2:', slope1, intercept1, r_value1**2)

df_fig2d = df_fig2.loc[(df_fig2['PAR']<1000) & (df_fig2['PAR']>500)]

#make aliases
Tair=df_fig2d['Tair']
Tsoil=df_fig2d['Tsoil']
NEE=df_fig2d['NEE']
VPD=df_fig2d['VPD']

#because the marker argument in scatter() doesn't accept a long list the same size of the data where each entry specifies the marker of that point (unlike c - color arg),
#we have to split the dataset into the categories and plot it over each other with multiple instances of scatter()

#separate data:
#important! check if lengths of each of these are the same
fig2d_cat1=df_fig2d.loc[(VPD>=0)&(VPD<=5)]
fig2d_cat2=df_fig2d.loc[(VPD>5)&(VPD<=10)]
fig2d_cat3=df_fig2d.loc[(VPD>10)&(VPD<=15)]
fig2d_cat4=df_fig2d.loc[VPD>15]

# visual reference for segments and boundaries:  |-(10-15)--|---(15-20)--|--(20-25)--|-(25+)---
# meaning 4 segments with 4 boundaries (upper segment unbounded: 25+ but lower segment bounded: 10 or greater

#colorbar settings
#for some reason the object created and assigned to cmap (matplotlib.colors.LinearSegmentedColormap) does not have an attribute .colors (which it should have? or maybe it's just meant to be a constructor?
#as such, to properly assign the 'outside range' cbar colors I'd have to either do a workaround, or I can just manually asign my colors. I'm opting for the latter.
colors=['#3A4CC0', '#AAC6FD', '#F6B79C', '#B30326']
cmap= mpl.colors.ListedColormap(colors, name='coolwarm_4', N=None)
colors_cbar=['#3A4CC0', '#AAC6FD', '#F6B79C', '#F6B79C']
cmap_cbar= mpl.colors.ListedColormap(colors_cbar, name='coolwarm_4', N=None) #have to do this weird trick to get the single-side unbounded cbar to work
cmap_cbar.set_over('#B30326')

#cols = [mpl.colors.to_hex(cmap(0)),mpl.colors.to_hex(cmap(1)),mpl.colors.to_hex(cmap(2)),mpl.colors.to_hex(cmap(3))] #may be unnecessary, use cmap(0) cmap(1) cmap(2) etc

cbar_bounds = [0, 5 ,10, 15 ] #boundaries excluding the abs min and abs max
norm = mpl.colors.BoundaryNorm(cbar_bounds, cmap.N)

fig,ax = plt.subplots()
ax_cbar = fig.add_axes([0.05, -0.20, 0.9, 0.05]) #coords are in proportion to fig size (fraction of fig dimension). 0,0 being bottom left corner, 1,1 being top right. format is [x1,y1,x2,y2] where x1,y1 is origin point of axes, and x2,y2 is the width and height of axes, respectively.

p1=ax.scatter(fig2d_cat1['Tsoil'],fig2d_cat1['NEE'],marker='.',s=5,c=colors[0],alpha=0.3) #scatter(x,y)
p2=ax.scatter(fig2d_cat2['Tsoil'],fig2d_cat2['NEE'],marker='s',s=5,c=colors[1],alpha=0.3) #scatter(x,y)
p3=ax.scatter(fig2d_cat3['Tsoil'],fig2d_cat3['NEE'],marker='d',s=5,c=colors[2],alpha=0.3) #scatter(x,y)
p4=ax.scatter(fig2d_cat4['Tsoil'],fig2d_cat4['NEE'],marker='+',s=10,c=colors[3],alpha=0.3) #scatter(x,y)

#trendlines (work in progress)
#ax.axline((0.0,intercept1),slope=slope1,c='black',label='reference') 
#ax.axline ((0.0,0.0), slope=1, c='r',linestyle='dashed',label='1:1')

ax.set_xlim(0,20)
ax.set_ylim(-35,25)
ax.set_ylabel(r'NEE [$\mu molm^{-2}s^{-1}$] (?)')
ax.set_xlabel(r'Tsoil [$\degree$C]')
#fig.suptitle('Correlation of increased VPD to reference run for An, \n (2008-2017 during growth seasons May-Oct.)')
ax.set_title('Fig2d: NEE vs Tsoil (color by VPD), 500<PAR<1000')
#ax.legend(loc='upper left')
#fig.subplots_adjust(top=0.87) #fix the top margin text overlap  

cb = mpl.colorbar.ColorbarBase(ax_cbar, cmap=cmap_cbar,
                                norm=norm,
                                # to use 'extend', you must specify two extra boundaries:
                                boundaries=cbar_bounds+[15],
                                extend='max', #use 'both' for left and right
                                ticks=cbar_bounds,  # optional
                                spacing='proportional',
                                orientation='horizontal')
cb.set_label('VPD')
#cbar=plt.colorbar(p1,ax=ax)
#cbar.ax.set_ylabel('Tair')
fig.set_figwidth(6)
fig.set_figheight(2.6)
fig.set_dpi(300)
plt.plot()

### end of figure 2c and d

### plot figure 2 a-f all in one

In [ ]:
#fig 2 a-f)

#slope1, intercept1, r_value1, p_value1, std_err1 = stats.linregress(df_tmp1['sim_data_1'], df_tmp1['sim_data_2'])  #linregres x, y . note r_value is Pearson's coefficient. R^2 is r_value**2
#print('slope, intercept, R2:', slope1, intercept1, r_value1**2)

df_fig2_PAR10 = df_fig2.loc[df_fig2['PAR']<1000]
df_fig2_PAR5_10 = df_fig2.loc[(df_fig2['PAR']<1000) & (df_fig2['PAR']>500)]

#make aliases
Tair_10=df_fig2_PAR10['Tair']
Tsoil_10=df_fig2_PAR10['Tsoil']
NEE_10=df_fig2_PAR10['NEE']
VPD_10=df_fig2_PAR10['VPD']

Tair5_10=df_fig2_PAR5_10['Tair']
Tsoil5_10=df_fig2_PAR5_10['Tsoil']
NEE5_10=df_fig2_PAR5_10['NEE']
VPD5_10=df_fig2_PAR5_10['VPD']

#because the marker argument in scatter() doesn't accept a long list the same size of the data where each entry specifies the marker of that point (unlike c - color arg),
#we have to split the dataset into the categories and plot it over each other with multiple instances of scatter()

#separate data:
#important! check if lengths of each of these are the same
fig2ac_cat1=df_fig2_PAR10.loc[(VPD_10>=0)&(VPD_10<=5)]
fig2ac_cat2=df_fig2_PAR10.loc[(VPD_10>5)&(VPD_10<=10)]
fig2ac_cat3=df_fig2_PAR10.loc[(VPD_10>10)&(VPD_10<=15)]
fig2ac_cat4=df_fig2_PAR10.loc[VPD_10>15]

fig2bd_cat1=df_fig2_PAR5_10.loc[(VPD5_10>=0)&(VPD5_10<=5)]
fig2bd_cat2=df_fig2_PAR5_10.loc[(VPD5_10>5)&(VPD5_10<=10)]
fig2bd_cat3=df_fig2_PAR5_10.loc[(VPD5_10>10)&(VPD5_10<=15)]
fig2bd_cat4=df_fig2_PAR5_10.loc[VPD5_10>15]

fig2e_cat1=df_fig2_PAR10.loc[(Tair_10>=10)&(Tair_10<=15)]
fig2e_cat2=df_fig2_PAR10.loc[(Tair_10>15)&(Tair_10<=20)]
fig2e_cat3=df_fig2_PAR10.loc[(Tair_10>20)&(Tair_10<=25)]
fig2e_cat4=df_fig2_PAR10.loc[Tair_10>25]

fig2f_cat1=df_fig2_PAR5_10.loc[(Tair5_10>=10)&(Tair5_10<=15)]
fig2f_cat2=df_fig2_PAR5_10.loc[(Tair5_10>15)&(Tair5_10<=20)]
fig2f_cat3=df_fig2_PAR5_10.loc[(Tair5_10>20)&(Tair5_10<=25)]
fig2f_cat4=df_fig2_PAR5_10.loc[Tair5_10>25]


# visual reference for segments and boundaries:  |-(10-15)--|---(15-20)--|--(20-25)--|-(25+)---
# meaning 4 segments with 4 boundaries (upper segment unbounded: 25+ but lower segment bounded: 10 or greater

#colorbar settings
#for some reason the object created and assigned to cmap (matplotlib.colors.LinearSegmentedColormap) does not have an attribute .colors (which it should have? or maybe it's just meant to be a constructor?
#as such, to properly assign the 'outside range' cbar colors I'd have to either do a workaround, or I can just manually asign my colors. I'm opting for the latter.
colors=['#3A4CC0', '#AAC6FD', '#F6B79C', '#B30326']
cmap= mpl.colors.ListedColormap(colors, name='coolwarm_4', N=None)
colors_cbar=['#3A4CC0', '#AAC6FD', '#F6B79C', '#F6B79C']
cmap_cbar= mpl.colors.ListedColormap(colors_cbar, name='coolwarm_4', N=None) #have to do this weird trick to get the single-side unbounded cbar to work
cmap_cbar.set_over('#B30326')

#cols = [mpl.colors.to_hex(cmap(0)),mpl.colors.to_hex(cmap(1)),mpl.colors.to_hex(cmap(2)),mpl.colors.to_hex(cmap(3))] #may be unnecessary, use cmap(0) cmap(1) cmap(2) etc

cbar_bounds_abcd = [0, 5 ,10, 15 ] #boundaries excluding the abs min and abs max
cbar_bounds_ef = [10, 15 ,20, 25 ]
norm_abcd = mpl.colors.BoundaryNorm(cbar_bounds_abcd, cmap.N)
norm_ef = mpl.colors.BoundaryNorm(cbar_bounds_ef, cmap.N)

#create figure
fig = plt.figure(layout='constrained') #optional arg: facecolor='grey'
fig.set_dpi(300)
fig.suptitle('Figure 2')
fig.set_figwidth(12)
fig.set_figheight(7.8)

#figTnest[0].suptitle('500<PAR<1000')
#figTnest[1].suptitle('PAR<1000')
#figTnestL,figtnestR = figTnest[0].subfigures(nrows=1, ncols=2)
axs=fig.subplot_mosaic([['A panel', 'B panel'],
                            ['A panel', 'B panel'],
                            ['C panel', 'D panel'],
                            ['C panel', 'D panel'],
                            ['cbar1',   'cbar1'  ],
                            ['E panel', 'F panel'],
                            ['E panel', 'F panel'],
                            ['cbar2',   'cbar2'  ],])


# Draw figure
# a) top left
p1a=axs['A panel'].scatter(fig2ac_cat1['Tair'],fig2ac_cat1['NEE'],marker='.',s=5,c=colors[0],alpha=0.3) #scatter(x,y) . fig2ac_cat1 contains values for both a and c
p2a=axs['A panel'].scatter(fig2ac_cat2['Tair'],fig2ac_cat2['NEE'],marker='s',s=5,c=colors[1],alpha=0.3)
p3a=axs['A panel'].scatter(fig2ac_cat3['Tair'],fig2ac_cat3['NEE'],marker='d',s=5,c=colors[2],alpha=0.3)
p4a=axs['A panel'].scatter(fig2ac_cat4['Tair'],fig2ac_cat4['NEE'],marker='+',s=10,c=colors[3],alpha=0.3)

# b) top right
p1b=axs['B panel'].scatter(fig2bd_cat1['Tair'],fig2bd_cat1['NEE'],marker='.',s=5,c=colors[0],alpha=0.3)
p2b=axs['B panel'].scatter(fig2bd_cat2['Tair'],fig2bd_cat2['NEE'],marker='s',s=5,c=colors[1],alpha=0.3)
p3b=axs['B panel'].scatter(fig2bd_cat3['Tair'],fig2bd_cat3['NEE'],marker='d',s=5,c=colors[2],alpha=0.3)
p4b=axs['B panel'].scatter(fig2bd_cat4['Tair'],fig2bd_cat4['NEE'],marker='+',s=10,c=colors[3],alpha=0.3)

# c) mid left
p1c=axs['C panel'].scatter(fig2ac_cat1['Tsoil'],fig2ac_cat1['NEE'],marker='.',s=5,c=colors[0],alpha=0.3)
p2c=axs['C panel'].scatter(fig2ac_cat2['Tsoil'],fig2ac_cat2['NEE'],marker='s',s=5,c=colors[1],alpha=0.3)
p3c=axs['C panel'].scatter(fig2ac_cat3['Tsoil'],fig2ac_cat3['NEE'],marker='d',s=5,c=colors[2],alpha=0.3)
p4c=axs['C panel'].scatter(fig2ac_cat4['Tsoil'],fig2ac_cat4['NEE'],marker='+',s=10,c=colors[3],alpha=0.3)

# d) mid right
p1d=axs['D panel'].scatter(fig2bd_cat1['Tsoil'],fig2bd_cat1['NEE'],marker='.',s=5,c=colors[0],alpha=0.3)
p2d=axs['D panel'].scatter(fig2bd_cat2['Tsoil'],fig2bd_cat2['NEE'],marker='s',s=5,c=colors[1],alpha=0.3)
p3d=axs['D panel'].scatter(fig2bd_cat3['Tsoil'],fig2bd_cat3['NEE'],marker='d',s=5,c=colors[2],alpha=0.3)
p4d=axs['D panel'].scatter(fig2bd_cat4['Tsoil'],fig2bd_cat4['NEE'],marker='+',s=10,c=colors[3],alpha=0.3)

# e) bottom leftVPD
p1e=axs['E panel'].scatter(fig2e_cat1['VPD'],fig2e_cat1['NEE'],marker='.',s=5,c=colors[0],alpha=0.3)
p2e=axs['E panel'].scatter(fig2e_cat2['VPD'],fig2e_cat2['NEE'],marker='s',s=5,c=colors[1],alpha=0.3)
p3e=axs['E panel'].scatter(fig2e_cat3['VPD'],fig2e_cat3['NEE'],marker='d',s=5,c=colors[2],alpha=0.3)
p4e=axs['E panel'].scatter(fig2e_cat4['VPD'],fig2e_cat4['NEE'],marker='+',s=10,c=colors[3],alpha=0.3)

# f) bottom right
p1f=axs['F panel'].scatter(fig2f_cat1['VPD'],fig2f_cat1['NEE'],marker='.',s=5,c=colors[0],alpha=0.3)
p2f=axs['F panel'].scatter(fig2f_cat2['VPD'],fig2f_cat2['NEE'],marker='s',s=5,c=colors[1],alpha=0.3)
p3f=axs['F panel'].scatter(fig2f_cat3['VPD'],fig2f_cat3['NEE'],marker='d',s=5,c=colors[2],alpha=0.3)
p4f=axs['F panel'].scatter(fig2f_cat4['VPD'],fig2f_cat4['NEE'],marker='+',s=10,c=colors[3],alpha=0.3)

axs['A panel'].set_title('500<PAR<1000')
axs['B panel'].set_title('PAR<1000')


#trendlines (work in progress)
#ax.axline((0.0,intercept1),slope=slope1,c='black',label='reference') 
#ax.axline ((0.0,0.0), slope=1, c='r',linestyle='dashed',label='1:1')

# Axes limits, titles, layout
axs['A panel'].set_ylim(-30,25) #a)
axs['A panel'].set_xlim(-15,40)
axs['B panel'].set_ylim(-30,25) #b)
axs['B panel'].set_xlim(-15,40)
axs['C panel'].set_ylim(-30,25) #c)
axs['C panel'].set_xlim(0,20)
axs['D panel'].set_ylim(-30,25) #d)
axs['D panel'].set_xlim(0,20)
axs['E panel'].set_ylim(-40,20) #e)
axs['E panel'].set_xlim(0,40)
axs['F panel'].set_ylim(-40,20) #f)
axs['F panel'].set_xlim(0,40)

#ax1.set_ylabel(r'NEE [$\mu molm^{-2}s^{-1}$] (?)')
#ax1.set_xlabel(r'Tair [$\degree$C]')
#ax1.set_title('Fig2a: NEE vs Tair (color by VPD), PAR<1000')
#ax.legend(loc='upper left')
#fig.subplots_adjust(top=0.87) #fix the top margin text overlap  

cb_abcd = mpl.colorbar.ColorbarBase(axs['cbar1'], cmap=cmap_cbar,
                                norm=norm_abcd,
                                # to use 'extend', you must specify two extra boundaries:
                                boundaries=cbar_bounds_abcd+[15],
                                extend='max', #use 'both' for left and right
                                ticks=cbar_bounds,  # optional
                                spacing='proportional',
                                orientation='horizontal')
axs['cbar1'].set_position([0,0.43,1,0.02])
axs['cbar1'].set_xlabel('VPD')


cb_ef = mpl.colorbar.ColorbarBase(axs['cbar2'], cmap=cmap_cbar,
                                norm=norm_ef,
                                # to use 'extend', you must specify two extra boundaries:
                                boundaries=cbar_bounds_ef+[25],
                                extend='max', #use 'both' for left and right
                                ticks=cbar_bounds,  # optional
                                spacing='proportional',
                                orientation='horizontal')
axs['cbar2'].set_position([0,0.07,1,0.02])
axs['cbar2'].set_xlabel('Tair')

#cbar=plt.colorbar(p1,ax=ax)
#cbar.ax.set_ylabel('Tair')

plt.plot()

In [ ]:
a_Test=axs['A panel']

In [ ]:
a_Test.set_position

In [ ]:
#backup of old way of doing it
#create figure
fig = plt.figure(layout='constrained') #optional arg: facecolor='grey'
fig.set_dpi(300)
fig.suptitle('Figure 2')
fig.set_figwidth(12)
fig.set_figheight(7.8)

figT, figB = fig.subfigures(nrows=2, ncols=1, height_ratios=[2,1],wspace=0.30) # dont know why but if I add the args "wspace=0.2,squeeze=False," then the outuputs are wrapped in arrays
figT.suptitle('figT')
figT.set_facecolor('coral')
figB.suptitle('figB')
figB.set_facecolor('red')

figTnest = figT.subfigures(nrows=1, ncols=1)

#figTnest[0].suptitle('500<PAR<1000')
#figTnest[1].suptitle('PAR<1000')
#figTnestL,figtnestR = figTnest[0].subfigures(nrows=1, ncols=2)
axT=figTnest.subplot_mosaic([['A panel', 'B panel'],
                            ['A panel', 'B panel'],
                            ['C panel', 'D panel'],
                            ['C panel', 'D panel'],
                            ['edge',     'edge']])



#axT_R=figTnest[1].subplots(nrows=2, ncols=1)

#axT=figTnest[0].subplots(nrows=2, ncols=2)
#axT.title('axT')
#figTnest[0].colorbar(pc, ax=axT)

figBnest = figB.subfigures(nrows=1, ncols=1)
figBnest.suptitle('figBnest')
axB=figBnest.subplot_mosaic([['A panel', 'B panel'],
                                ['A panel', 'B panel'],
                                ['edge',     'edge']])


#axB=figBnest.subplot_mosaic(nrows=1,ncols=2)
#axB.title('axB')
#figBnest.colorbar(pc, ax=axB)


### end of figure 2 all in one

## figure 3

### end of figure 3

In [ ]:
help(figT.set_figure)